In [85]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import sent_tokenize

from time import time
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation

from nltk.tag import pos_tag
from textblob import TextBlob

import re
import copy
import operator

import pickle
import ast
import pyLDAvis.sklearn

In [4]:
tags=pd.read_csv('../../../local_files/Projects/Instagram_Likes_Local/data/cat_tags.csv')
cat_meta=pd.read_csv('../../../local_files/Projects/Instagram_Likes_Local/data/cat_hard_mean.csv')

In [7]:
tags.head(2)

,image_urls,labels,label_probs
0,http://scontent.cdninstagram.com/t51.2885-15/s...,"['cat', 'cute', 'kitten', 'pet', 'portrait', '...","[0.99867606, 0.99583024, 0.98985684, 0.9892707..."
1,http://scontent.cdninstagram.com/t51.2885-15/s...,"['cat', 'animal', 'mammal', 'pet', 'cute', 'po...","[0.99749255, 0.98964846, 0.9838639, 0.9806832,..."


In [6]:
cat_meta.head(2)

,caption,user_url,username,comments,likes,image_urls,rating,name,pic_path,checksum
0,/\m/\ONSTERs in closets are real 🚪😈 @algonquin...,http://www.imgrum.org/user/princessmonstertruc...,princessmonstertruck,23.0,4661.0,http://scontent.cdninstagram.com/t51.2885-15/s...,1.0,PrincessMonsterTruck ©,0cc6b49841d86de62f7f75742328ebcfa2ec04ef.jpg,6ec7173ba299c5471fd6c16920b0efbf
1,Thanks again to our awesome friends @littergen...,http://www.imgrum.org/user/princessmonstertruc...,princessmonstertruck,10.0,2172.0,http://scontent.cdninstagram.com/t51.2885-15/s...,0.0,PrincessMonsterTruck ©,d7517fa9d348e9802ecc2140d35f25f43ba3769e.jpg,313ddc6627520e45c0821e1c058bedb4


In [8]:
cats = pd.merge(tags,cat_meta, how = 'left', on='image_urls')

In [9]:
cats.head(2)

,image_urls,labels,label_probs,caption,user_url,username,comments,likes,rating,name,pic_path,checksum
0,http://scontent.cdninstagram.com/t51.2885-15/s...,"['cat', 'cute', 'kitten', 'pet', 'portrait', '...","[0.99867606, 0.99583024, 0.98985684, 0.9892707...",These hips don't lie. #cattitude,http://www.imgrum.org/user/cobythecat/2126993164,cobythecat,160.0,51343.0,1.0,Coby The Cat,c9158cf4776223e00f972911df9c1e50b20da6ac.jpg,6099b4bb85e1d59547a4053fb43c286c
1,http://scontent.cdninstagram.com/t51.2885-15/s...,"['cat', 'animal', 'mammal', 'pet', 'cute', 'po...","[0.99749255, 0.98964846, 0.9838639, 0.9806832,...",May I help you?\n#cat #cats #neko #猫 #katze #...,http://www.imgrum.org/user/emonemon/49896992,emonemon,15.0,766.0,0.0,Satoru Konno,05580d2b7094bd3ed181ab139529217eea33597f.jpg,f362e7f2eb780f2ac36bcd61aa85f2d0


In [42]:
def getList(row):
    label1 = ast.literal_eval(row)#cats.ix[row,'labels']
    return ' '.join(word for word in label1)

In [55]:
cats['tag_sentences'] = cats['labels'].apply(getList)

In [56]:
cats.head(2)

,image_urls,labels,label_probs,caption,user_url,username,comments,likes,rating,name,pic_path,checksum,tag_sentences
0,http://scontent.cdninstagram.com/t51.2885-15/s...,"['cat', 'cute', 'kitten', 'pet', 'portrait', '...","[0.99867606, 0.99583024, 0.98985684, 0.9892707...",These hips don't lie. #cattitude,http://www.imgrum.org/user/cobythecat/2126993164,cobythecat,160.0,51343.0,1.0,Coby The Cat,c9158cf4776223e00f972911df9c1e50b20da6ac.jpg,6099b4bb85e1d59547a4053fb43c286c,cat cute kitten pet portrait eye domestic mamm...
1,http://scontent.cdninstagram.com/t51.2885-15/s...,"['cat', 'animal', 'mammal', 'pet', 'cute', 'po...","[0.99749255, 0.98964846, 0.9838639, 0.9806832,...",May I help you?\n#cat #cats #neko #猫 #katze #...,http://www.imgrum.org/user/emonemon/49896992,emonemon,15.0,766.0,0.0,Satoru Konno,05580d2b7094bd3ed181ab139529217eea33597f.jpg,f362e7f2eb780f2ac36bcd61aa85f2d0,cat animal mammal pet cute portrait kitten str...


In [60]:
good_cats = cats[cats['rating']==1]
ngood_cats = cats[cats['rating']==0]

In [61]:
good_cats = good_cats['tag_sentences'].tolist()
ngood_cats = ngood_cats['tag_sentences'].tolist()

In [66]:
def top_words(model, feature_names, n_top_words):
    lst = []
    for topic_idx, topic in enumerate(model.components_):
#         lst.append(topic_idx)
        lst.append(" , ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    return lst

nmf = NMF(n_components=5, random_state=1,
              alpha=.1, l1_ratio=.3).fit(tfidf)


In [ ]:
topWords = top_words(nmf, tfidf_feature_names, 5)

In [93]:
vect = TfidfVectorizer(stop_words="english",lowercase=False)
mat = vect.fit_transform(good_cats).toarray()



[('Carnivora', 0),
 ('Christmas', 1),
 ('Easter', 2),
 ('Gothic', 3),
 ('H2O', 4),
 ('Halloween', 5),
 ('International', 6),
 ('Jurassic', 7),
 ('New', 8),
 ('Siberian', 9),
 ('Thanksgiving', 10),
 ('Venetian', 11),
 ('Web', 12),
 ('Wide', 13),
 ('World', 14),
 ('Year', 15),
 ('abstract', 16),
 ('accessory', 17),
 ('accident', 18),
 ('accomplishment', 19),
 ('ace', 20),
 ('achievement', 21),
 ('action', 22),
 ('active', 23),
 ('actor', 24),
 ('actress', 25),
 ('administration', 26),
 ('adolescent', 27),
 ('adorable', 28),
 ('adult', 29),
 ('adventure', 30),
 ('advertising', 31),
 ('affection', 32),
 ('aggressive', 33),
 ('agility', 34),
 ('agriculture', 35),
 ('aid', 36),
 ('aircraft', 37),
 ('airplane', 38),
 ('airport', 39),
 ('alcohol', 40),
 ('alert', 41),
 ('alphabet', 42),
 ('ancient', 43),
 ('angry', 44),
 ('animal', 45),
 ('annual', 46),
 ('antique', 47),
 ('apartment', 48),
 ('apparel', 49),
 ('apple', 50),
 ('appliance', 51),
 ('arch', 52),
 ('architecture', 53),
 ('arrow', 5

In [82]:
tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                               stop_words = 'english',
                               lowercase = True,
                               token_pattern = r'\b[a-zA-Z]{3,}\b',
                               max_df = 0.5,
                               min_df = 2)
dtm_tf = tf_vectorizer.fit_transform(good_cats)
lda_tf = LatentDirichletAllocation(n_topics=5, random_state=0, n_jobs = -1)
lda_tf.fit(dtm_tf)
# data = pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer)
# pyLDAvis.display(data)

/Users/bauer/anaconda3/lib/python3.5/site-packages/sklearn/decomposition/online_lda.py:508: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7, learning_method=None,
             learning_offset=10.0, max_doc_update_iter=100, max_iter=10,
             mean_change_tol=0.001, n_jobs=-1, n_topics=5, perp_tol=0.1,
             random_state=0, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [ ]:
tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                               stop_words = 'english',
                               lowercase = True,
                               token_pattern = r'\b[a-zA-Z]{3,}\b',
                               max_df = 0.5,
                               min_df = 2)
dtm_tf = tf_vectorizer.fit_transform(ngood_cats)
lda_tf = LatentDirichletAllocation(n_topics=5, random_state=0, n_jobs = -1)
lda_tf.fit(dtm_tf)
# data = pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer)
# pyLDAvis.display(data)

/Users/bauer/anaconda3/lib/python3.5/site-packages/sklearn/decomposition/online_lda.py:508: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


In [192]:
def nmfTopics(data, n_topics, n_top_words, n_features):

    # Use tf-idf features for NMF.
    print("Extracting tf-idf features for NMF...")
    tfidf_vectorizer = TfidfVectorizer(min_df=1, max_df=.8,
                                       max_features = 600,
                                       stop_words='english')
    t0 = time()
    tfidf = tfidf_vectorizer.fit_transform(data)
    print("done in %0.3fs." % (time() - t0))

    # Fit the NMF model
    print("Fitting the NMF model with tf-idf features, "
          "n_samples=%d and n_features=%d..."
          % (int(len(data)), int(len(data)/2)))
    t0 = time()
    nmf = NMF(n_components=n_topics, random_state=1,
              alpha=.1, l1_ratio=.3).fit(tfidf)
    print("done in %0.3fs." % (time() - t0))

    print("\nTopics in NMF model:")
    tfidf_feature_names = tfidf_vectorizer.get_feature_names()
    return top_words(nmf, tfidf_feature_names, n_top_words), nmf.fit_transform(tfidf)

In [193]:
nmfTopics(good_cats, 1, 10, len(good_cats))

Extracting tf-idf features for NMF...
done in 0.086s.
Fitting the NMF model with tf-idf features, n_samples=4169 and n_features=2084...
done in 0.059s.

Topics in NMF model:


(['whisker , eye , young , looking , domestic , downy , little , funny , sit , adorable'],
 array([[ 0.06683872],
        [ 0.0059519 ],
        [ 0.00016183],
        ..., 
        [ 0.02810042],
        [ 0.06857986],
        [ 0.04827133]]))

In [194]:
nmfTopics(ngood_cats, 1, 10, len(ngood_cats))

Extracting tf-idf features for NMF...
done in 0.126s.
Fitting the NMF model with tf-idf features, n_samples=5821 and n_features=2910...
done in 0.088s.

Topics in NMF model:


(['eye , kitten , fur , domestic , downy , little , pet , young , looking , cute'],
 array([[ 0.06652533],
        [ 0.06870907],
        [ 0.0847269 ],
        ..., 
        [ 0.01256018],
        [ 0.00686939],
        [ 0.08553935]]))